In [58]:
# Python
from pathlib import Path
import json

# Database  
import sqlite3

# Data analysis
import pandas as pd

# Geospatial
import geopandas as gpd
import shapely.wkb
from shapely.wkb import loads

In [4]:
current_path = Path()

In [32]:
# Database
conn = sqlite3.connect(current_path / 'data' / 'caf_accessibility.db')

In [91]:
bogota_cuenca_df_parquet = pd.read_parquet(
    current_path / 'data' / 'bogota_cuenca_v2.parquet'
)

bogota_cuenca_gdf_geo = gpd.read_parquet(
    current_path / 'data' / 'bogota_cuenca_geo_v2.parquet',
)

In [96]:
bogota_cuenca_df = pd.concat([bogota_cuenca_df_parquet, bogota_cuenca_gdf_geo[['geometry']]], axis=1)

In [113]:
hospitales_gdf = gpd.read_parquet(
    current_path / 'data' / 'Bogota_SaluHosp.parquet'
)

In [117]:
espacios_verdes_gdf = gpd.read_parquet(
    current_path / 'data' / 'Bogota_EspaVerd.parquet'
)

In [115]:
def save_geo_in_sqlite_db(conn:sqlite3.Connection, table_name, geo_dict_json):
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Geos'")
    result = c.fetchone()
    if not result:
        c.execute('''CREATE TABLE Geos (id INTEGER PRIMARY KEY AUTOINCREMENT, table_ TEXT UNIQUE, geometry TEXT)''')
    c.execute('''INSERT INTO Geos (table_, geometry) VALUES (?, ?)''', (table_name, geo_dict_json))
    conn.commit()

def save_dataframe_in_sqlite_db(df:pd.DataFrame, table_name:str, conn:sqlite3.Connection):
    print(type(df))
    if 'geometry' not in df.columns:
        raise ValueError('Dataframe must have a geometry column')
    # df['geometry'] = df.apply(lambda x: shapely.wkb.dumps(x.geometry), axis=1)
    bogota_cuenca_df_geo = gpd.GeoDataFrame(df[['geometry']], geometry='geometry')
    geo_dict_json = json.dumps(bogota_cuenca_df_geo[['geometry']].__geo_interface__)
    save_geo_in_sqlite_db(conn, table_name, geo_dict_json)
    df.drop('geometry', axis=1).to_sql(table_name, conn, if_exists='replace', index=False)



In [ ]:
save_dataframe_in_sqlite_db(df=bogota_cuenca_df, table_name='Accessibility', conn=conn)

In [116]:
save_dataframe_in_sqlite_db(df=hospitales_gdf, table_name='Hospitals', conn=conn)

<class 'geopandas.geodataframe.GeoDataFrame'>


In [118]:
save_dataframe_in_sqlite_db(df=espacios_verdes_gdf, table_name='Green_Areas', conn=conn)

<class 'geopandas.geodataframe.GeoDataFrame'>
